In [1]:
import numpy as np
from sklearn.datasets import load_iris, load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
import sklearn

In [2]:
import redisai as rai

In [3]:
con = rai.Client(host='localhost', port=6379)

In [10]:
iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y)

# train a model
model = LogisticRegression(max_iter=5000)
model.fit(X_train, y_train)

# convert the model to ONNX
initial_types = [
  ('input', FloatTensorType([None, 4]))
]

onnx_model = convert_sklearn(model, initial_types=initial_types)

# save the model
with open("iris.onnx", "wb") as f:
  f.write(onnx_model.SerializeToString())

In [11]:
model = open('./iris.onnx', 'rb').read()

In [13]:
con.modelset(key="onnx_model", backend='ONNX', device='CPU', data=model, outputs=['classes', 'confidence'])

'OK'

In [14]:
con.modelscan()

/home/kkalyagina/anaconda3/envs/redisai/lib/python3.7/site-packages/redisai/client.py:336: UserWarning: Experimental: Model List API is experimental and might change in the future without any notice
  "in the future without any notice", UserWarning)


[['onnx_model', '']]

In [15]:
con.tensorset(key="input", tensor=[5.0, 3.4, 1.6, 0.4, 6.0, 2.2, 5.0, 1.5], shape=[2,4], dtype='float')

'OK'

In [16]:
con.tensorget(key='input')

array([[5. , 3.4, 1.6, 0.4],
       [6. , 2.2, 5. , 1.5]], dtype=float32)

In [17]:
con.tensorget(key='input', meta_only=True)

{'dtype': 'FLOAT', 'shape': [2, 4]}

In [18]:
con.tensorget(key='input', as_numpy=False)

{'dtype': 'FLOAT',
 'shape': [2, 4],
 'values': [5.0,
  3.4000000953674316,
  1.600000023841858,
  0.4000000059604645,
  6.0,
  2.200000047683716,
  5.0,
  1.5]}

In [19]:
con.modelrun(key="onnx_model", inputs="input", outputs=['class', 'confidence'])

'OK'

In [20]:
con.tensorget(key='class')

array([0, 2])

In [21]:
con.tensorget(key='class', as_numpy=True, meta_only=True)

{'dtype': 'INT64', 'shape': [2]}

In [23]:
con.tensorset(key='predict', tensor=[5.0, 3.4, 1.6, 0.4], shape=[1,4], dtype='float')
con.modelrun(key='onnx_model', inputs='predict', outputs=['class', 'confidence'])
con.tensorget(key='class')[0]

0